# COVID-19 Prediction

In [17]:
# import py packages
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import summary

from tqdm import tqdm

import math
import os
import csv


## Dataset 

In [18]:
class CovidDataset(Dataset):
    def __init__(self, x, y) -> None:
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, index):
        if self.y is None:
            return self.x[index]
        else:
            return self.x[index], self.y[index]
        
    def __len__(self):
        return len(self.x)   

## nn Model

In [19]:
class PredModel(nn.Module):
    def __init__(self,input_dim) -> None:
        super(PredModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
    
    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

## DataLoader

In [20]:
train_df = pd.read_csv("./dataset/covid.train.csv")
test_df = pd.read_csv("./dataset/covid.test.csv")
train_df

,id,AL,AK,AZ,AR,CA,CO,CT,FL,GA,...,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4,tested_positive.4
0,0,0,0,0,0,0,0,0,1,0,...,31.113209,67.394551,36.674291,40.743132,17.842221,4.093712,10.440071,8.627117,37.329512,7.456154
1,1,0,0,0,0,0,1,0,0,0,...,33.920257,64.398380,34.612238,44.035688,17.808103,4.924935,10.172662,9.954333,32.508881,8.010957
2,2,0,0,0,0,0,0,0,0,0,...,31.604604,62.101064,26.521875,36.746453,13.903667,7.313833,10.388712,7.956139,36.745588,2.906977
3,3,0,0,0,0,0,0,0,0,0,...,35.115738,67.935520,38.022492,48.434809,27.134876,3.101904,10.498683,8.231522,38.680162,12.575816
4,4,0,0,0,0,0,0,0,0,0,...,35.129714,69.934592,38.242368,49.095933,22.683709,4.594620,9.878927,9.469290,28.344123,21.428589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,2695,0,0,0,0,0,0,0,0,0,...,39.209278,65.815096,37.046827,46.777637,21.952499,2.724187,14.658436,11.099308,33.154910,13.437482
2695,2696,0,0,0,0,0,0,0,0,0,...,28.984491,66.126427,31.299807,40.399527,16.783327,14.058799,13.129031,10.492896,36.670282,5.040143
2696,2697,0,0,0,1,0,0,0,0,0,...,34.346100,63.456270,30.829009,41.828908,20.597714,2.391852,16.940896,12.888275,42.452451,13.618256
2697,2698,0,0,0,0,0,0,0,0,0,...,30.451071,67.477177,33.975333,45.837535,18.437260,3.088666,12.621144,10.096939,29.668802,9.512585


In [21]:
def set_seed(seed=0):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): # for cuda
        torch.cuda.manual_seed(seed)

    
def train_valid_split(dataset, valid_ratio=0.2, seed=0):
    total_num = len(dataset)
    split_num = int(np.floor(valid_ratio * total_num))
    generator = torch.Generator().manual_seed(seed)
    train_set, valid_set = random_split(dataset, [total_num-split_num, split_num], generator=generator)
    return np.array(train_set), np.array(valid_set)

config = {
    "seed": 0,
    "batch_size": 256,
    "lr": 1e-5,
    "epochs": 50,
    "save_path": "./models/model.pth",
}

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu" 
)

set_seed(config["seed"])

train_df = train_df.drop(columns=["id"])
train_data = train_df.values
test_data = test_df.values
train_data, valid_data = train_valid_split(train_data, valid_ratio=0.2, seed=config["seed"])

print(f"train_data: {train_data.shape}")
print(f"valid_data: {valid_data.shape}")
print(f"test_data: {test_data.shape}")



def select_features(data):
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

X_train, y_train = select_features(train_data)
X_valid, y_valid = select_features(valid_data)
X_test, y_test = select_features(test_data)


train_dataset = CovidDataset(X_train, y_train)
valid_dataset = CovidDataset(X_valid, y_valid)
test_dataset = CovidDataset(X_test, None)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=config["batch_size"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)



train_data: (2160, 117)
valid_data: (539, 117)
test_data: (1078, 117)


## Training

In [22]:
def train_loop(train_loader, valid_loader, model, config, device):
    loss_fn = nn.MSELoss(reduction="mean")
    optimizer = torch.optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)

    if not os.path.exists("./models"):
        os.makedirs("./models")
    
    n_epochs = config["epochs"]
    best_loss = float("inf")

    for epoch in range(n_epochs):
        model.train()

        train_pbar = tqdm(train_loader, position=0, leave=True)

        for X, y in train_pbar:
            optimizer.zero_grad()
            X = X.to(device)
            y = y.to(device)

            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()

            train_pbar.set_description(f"Epoch {epoch+1}/{n_epochs}")
            train_pbar.set_postfix({'loss': loss.item()})
            
        model.eval()
        valid_loss = []
        for X, y in valid_loader:
            X = X.to(device)
            y = y.to(device)
            with torch.no_grad():
                y_pred = model(X)
                loss = loss_fn(y_pred, y)
            valid_loss.append(loss.item())

        loss = np.mean(valid_loss)
        print(f"Epoch {epoch+1}/{n_epochs}, valid loss: {loss}")
        if loss < best_loss:
            best_loss = loss
            torch.save(model.state_dict(), config["save_path"])
            print(f"save model at epoch {epoch+1}, and with loss{best_loss}")

model = PredModel(input_dim=X_train.shape[1]).to(device)
train_loop(train_loader, valid_loader, model, config, device)

        

Epoch 1/50: 100%|██████████| 9/9 [00:00<00:00, 67.55it/s, loss=84.7]


Epoch 1/50, valid loss: 89.58153788248698
save model at epoch 1, and with loss89.58153788248698


Epoch 2/50: 100%|██████████| 9/9 [00:00<00:00, 171.33it/s, loss=45.6]


Epoch 2/50, valid loss: 54.99344380696615
save model at epoch 2, and with loss54.99344380696615


Epoch 3/50: 100%|██████████| 9/9 [00:00<00:00, 174.04it/s, loss=46]


...

Epoch 48/50, valid loss: 3.681119998296102
save model at epoch 48, and with loss3.681119998296102


Epoch 49/50: 100%|██████████| 9/9 [00:00<00:00, 180.34it/s, loss=3.54]


Epoch 49/50, valid loss: 3.568037509918213
save model at epoch 49, and with loss3.568037509918213


Epoch 50/50: 100%|██████████| 9/9 [00:00<00:00, 187.86it/s, loss=2.32]


Epoch 50/50, valid loss: 3.5257187684377036
save model at epoch 50, and with loss3.5257187684377036
